# PHY4268 Tutorial 4 - Qiskit VQE with active space

1. **S. G. Nana Engo**, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
2. **J-P. Tchapet Njafa**, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
       
May 2024

A la fin de ce tutoriel, l'apprenant doit être capable d'utiliser Qiskit-nature pour
1. calculer l'énergie de l'état fondamental d'un Hamiltonien quelconque;
2. définir l'espace actif d'un système moléculaire autour du niveau de Fermi (Niveau HOMO-LUMO);
3. calculer les énergies de l'état fondamental et des premiers états excités y afférent.


## Hamiltonien quelconque

On considère l'Hamiltonien
$$\mathtt{H} = 0.4\mathbb{I}\mathtt{X} + 0.6\mathbb{I}\mathtt{Z} + 0.8\mathtt{XY}$$

Pour un $|\psi\rangle$ donné nous voulons évaluer la valeur moyenne de cet Hamiltonien :

$$\langle \mathtt{H} \rangle = \langle \psi |\mathtt{H}| \psi \rangle = 0.4 \langle \psi | \mathbb{I}\mathtt{X} |\psi \rangle + 0.6 \langle \psi | \mathbb{I}\mathtt{Z} | \psi \rangle + 0.8 \langle \psi | \mathtt{XY} | \psi \rangle.$$

Comme on peut voir la valeur moyenne $\langle\mathtt{H} \rangle$ pourrait être calculée en ajoutant les valeurs moyennes de ses parties (termes de Pauli). L'algorithme fait exactement cela. Il construit un circuit quantique pour chaque terme de Pauli et calcule la valeur moyenne du terme de Pauli correspondant. Ensuite, l'algorithme additionne toutes les valeurs moyennes calculées des termes de Pauli et obtient la valeur moyenne de $\mathtt{H}$.


In [56]:
try:
    import qiskit_nature
except:
    %pip install qiskit-nature[pyscf] -U
    import qiskit_nature

qiskit_nature.__version__

'0.7.2'

In [57]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyMinimumEigensolver

H = SparsePauliOp.from_list([("IX", 0.4), ("IZ", 0.6), ("XY", 0.8)])
print(H)

exact_result = NumPyMinimumEigensolver().compute_minimum_eigenvalue(H)
print(f'\n The exact ground state energy is: {exact_result.eigenvalue}')

SparsePauliOp(['IX', 'IZ', 'XY'],
              coeffs=[0.4+0.j, 0.6+0.j, 0.8+0.j])

 The exact ground state energy is: -1.0770329614269012


Si on a un Hamiltonien
$$\mathtt{H} = a\mathbb{I} + b\mathtt{Z} + c\mathtt{X} + d\mathtt{Y},\qquad a,b,c,d\in\mathbb{R},$$
il faut faire appel à des nombres arbitraires pour a, b, c et d.

In [58]:
from numpy import random

a, b, c, d = 10*random.sample(4)

H = SparsePauliOp.from_list([("I", a), ("Z", b), ("X", c), ("Y", d)])

print(H)
exact_result = NumPyMinimumEigensolver().compute_minimum_eigenvalue(H)
print(f'\n The exact ground state energy is: {exact_result.eigenvalue}')

SparsePauliOp(['I', 'Z', 'X', 'Y'],
              coeffs=[9.41174256+0.j, 1.31125011+0.j, 6.8264229 +0.j, 7.8864318 +0.j])

 The exact ground state energy is: -1.1008800017614435


## Molécule d'eau

### Hamiltionien électronique

![Water_HF_references](Graphics/Water_HF_references.png)

La molécule d'eau a un nombre total de dix électrons. Le déterminant de Slater résultant de l'occupation des cinq premières orbitales moléculaires d'énergie la plus basse avec deux électrons *appariés* dans chaque orbitale, l'un avec spin-up et l'autre avec spin-down, est dit être un état HF à couche fermée avec une *multiplicité de spin=1*. Alternativement, si nous définissons une occupation où les quatre premières orbitales sont doublement occupées et les deux suivantes sont occupées individuellement par des électrons *non appariés* avec spin-up, on dit qu'il s'agit d'un état HF à couche ouverte avec une *multiplicité de spin=3*.

Il n'est pas superflue de rappeler à nouveau ce qui suit.

* La multiplicité, que nous pouvons définir comme $(N_{\rm unpaired}^e + 1)$ avec $N_{\rm unpaired}^e$ le nombre d'électrons non appariés, détermine l'occupation des orbitales moléculaires dans les calculs HF.

* Les orbitales moléculaires sont généralement représentées comme une combinaison linéaire de
**orbitales atomiques**. Les coefficients de d'expansion dans la base atomique sont
calculé en utilisant la méthode de Hartree-Fock (HF). Dans l'approximation HF, chaque électron de la molécule est traité comme une particule **indépendante** qui se déplace sous l'influence du Coulomb potentiel dû aux noyaux, et un champ moyen généré par tous les autres
électrons. Les coefficients optimisés sont précisément ce dont on a besoin pour
construire le Hamiltonien de la seconde quantification.

### Active-Space reduction

En général, les méthodes de corrélation d'électrons post-Hartree-Fock étendent la fonction d'état de la molécule autour de la solution Hartree-Fock, en ajoutant des déterminants de Slater, communément appelés **configurations**, qui résultent de l'excitation des électrons des orbitales HF occupées vers les orbitales inoccupées. Malgré le fait qu'il existe différentes techniques pour tronquer cette expansion, le nombre de configurations augmente de manière combinatoire avec le nombre d'électrons et de fonctions de base et la tâche de trouver les coefficients d'expansion de la fonction d'état devient numériquement insoluble si nous voulons inclure l'ensemble complet des orbitales moléculaires. .

Afin de contourner l'explosion combinatoire, nous pouvons créer un espace actif en classant les orbitales moléculaires en orbitales doublement occupées, actives et externes :

* les orbitales doublement occupées (**core orbitals**) sont toujours occupées par deux électrons;
* les orbitales actives (**valence orbitals**) peuvent être occupées par zéro, un ou deux électrons.
* les orbitales externes (**virtual orbitals**) ne sont jamais occupées.

![Sketch_active_space](Graphics/Sketch_active_space.png)

Dans cette approximation, un certain nombre d'*électrons actifs* peuvent peupler les *orbitales actives* à partir desquelles nous pouvons générer un espace de taille finie de déterminants de Slater.

**Note**</br>
Le nombre de *spin-orbitales actives* détermine le *nombre de qubits* requis pour effectuer des simulations quantiques de la structure électronique de la molécule.

Pour le cas de la molécule d'eau décrite à l'aide d'un ensemble de base minimal, nous avons un total de dix électrons occupant les cinq premières des sept orbitales moléculaires dans l'état de référence HF.

Avec `qiskit_nature.second_q.transformers.ActiveSpaceTransformer`, la réduction se fait en calculant l'opérateur de Fock inactif qui est défini comme
$$ F^I_{pq} = h_{pq} + \sum_i (2 g_{iipq} - g_{iqpi}),
$$
et l'énergie inactive qui est donnée par
$$ E^I = \sum_j h_{ji} + F^I_{jj} = \frac12 \sum_{ij} \Big(h_{ij} + F^I_{jj}\Big) D^I_{ij} ,$$
où $i$ et $j$ itèrent sur les orbitales inactives. En utilisant l'opérateur de Fock inactif à la place des intégrales à un électron, la description de l'espace actif contient un potentiel effectif généré par les électrons inactifs. Par conséquent, cette méthode permet l'exclusion des électrons non centraux tout en conservant une description de haute qualité du système.

In [59]:
try:
    import qiskit_nature
except:
    %pip install qiskit-nature[pyscf] -U
    import qiskit_nature

qiskit_nature.__version__

'0.7.2'

In [60]:
from qiskit_nature.second_q.drivers import PySCFDriver

H2O_driver = PySCFDriver(
    atom='O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0',
    basis="sto-3g"
)

H2O_problem = H2O_driver.run()
H2O_hamiltonian = H2O_problem.hamiltonian
H2O_coefficients = H2O_hamiltonian.electronic_integrals
H2O_fermionic_op = H2O_hamiltonian.second_q_op()


In [61]:
print(f'Molecule strcuture: {H2O_problem.molecule}')
print(f'Reference energy: {H2O_problem.reference_energy}')
print(f'Nuclear repulsion energy: {H2O_problem.nuclear_repulsion_energy}')
print(f'Number of particules: {H2O_problem.num_particles}')
print(f'Number of spatial orbitals: {H2O_problem.num_spatial_orbitals}')
print(f'Number of molecular orbitals: {H2O_problem.num_spin_orbitals}')
print(f'Number of alpha electrons: {H2O_problem.num_alpha}')
print(f'Number of beta electrons: {H2O_problem.num_beta}')
print(f'Occupations of the alpha-spin orbitals: {H2O_problem.orbital_occupations}')


Molecule strcuture: Molecule:
	Multiplicity: 1
	Charge: 0
	Unit: Bohr
	Geometry:
		O	(0.0, 0.0, 0.0)
		H	(1.430522676295752, 1.1073795089951262, 0.0)
		H	(-1.430522676295752, 1.1073795089951262, 0.0)
	Masses:
		O	16
		H	1
		H	1
Reference energy: -74.96294665653875
Nuclear repulsion energy: 9.193913160623385
Number of particules: (5, 5)
Number of spatial orbitals: 7
Number of molecular orbitals: 14
Number of alpha electrons: 5
Number of beta electrons: 5
Occupations of the alpha-spin orbitals: [1. 1. 1. 1. 1. 0. 0.]


In [62]:

print(f"HOMO and LUMO indexes are (Fermi surface) {H2O_problem.num_alpha-1, H2O_problem.num_alpha}")
print(f"HOMO and LUMO energies are {H2O_problem.orbital_energies[H2O_problem.num_alpha-1],H2O_problem.orbital_energies[H2O_problem.num_alpha]}")
print(f"HOMO-LUMO gap is {abs(H2O_problem.orbital_energies[H2O_problem.num_alpha-1]-H2O_problem.orbital_energies[H2O_problem.num_alpha])}")


HOMO and LUMO indexes are (Fermi surface) (4, 5)
HOMO and LUMO energies are (-0.3912423661263905, 0.6055762144356047)
HOMO-LUMO gap is 0.9968185805619952


### 1.9.3. <a id='toc1_9_3_'></a>[`FreezeCoreTransformer`](#toc0_)

Ce transformateur vous offre un moyen très simple de geler les _core orbitales_ du système moléculaire. Il nécessite que votre problème contienne l'attribut `.molecule` à partir duquel il peut extraire les informations atomiques nécessaires pour effectuer cette réduction d'espace de Hilbert.

Appliquons `qiskit_nature.second_q.transformers.FreezeCoreTransforme` à notre molécule qui, dans ce cas, supprimera la seule orbitale d'énergie la plus basse (réduisant le nombre total d'orbitales spatiales de 7 à 6) et supprimant également les deux électrons de l'intérieur de cette orbitale (comme reflété par le nombre modifié de particules).

In [63]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

fc_transformer = FreezeCoreTransformer()

fc_H2O_problem = fc_transformer.transform(H2O_problem)
print(f'Number of particules with FC: {fc_H2O_problem.num_particles}')
print(f'Number of spatial orbitals with FC: {fc_H2O_problem.num_spatial_orbitals}')


Number of particules with FC: (4, 4)
Number of spatial orbitals with FC: 6


Notez que cette transformation se traduira par un décalage d'énergie constant résultant de la suppression des électrons du noyau. Ce décalage est enregistré à l'intérieur de l'attribut `constants`" de l'Hamiltonien.

In [64]:
print(fc_H2O_problem.hamiltonian.constants)

{'nuclear_repulsion_energy': 9.193913160623385, 'FreezeCoreTransformer': -60.66177520598535}


Vous pouvez fournir une liste d'indices orbitaux (se rappeler qu'on commence par 0) qui doivent être supprimés du système.

> **Remarque :** Ces orbitales *doivent* être inoccupées, sinon vous subirez une erreur importante dans votre calcul. Vous devez absolument savoir quelles orbitales vous supprimez, car la suppression des mauvaises orbitales peut toujours entraîner de grandes erreurs si la dynamique des systèmes est modifiée de manière significative.

In [65]:
fc_H2O_transformer = FreezeCoreTransformer(remove_orbitals=[5, 6])

fc_H2O_problem = fc_H2O_transformer.transform(H2O_problem)
print(fc_H2O_problem.num_particles)
print(fc_H2O_problem.num_spatial_orbitals)

(4, 4)
4


### `ActiveSpaceTransformer`

Ce transformateur généralise la réduction de l'espace de Hilbert qui est effectuée par le `FreezeCoreTransformer`. En termes simples, il vous permet de spécifier un _active space_ en sélectionnant le nombre d'électrons actifs et le nombre d'orbitales spatiales actives. Selon ces paramètres, l'espace actif sera choisi autour du niveau de Fermi.

L'espace actif peut être configuré de l'une des manières suivantes via l'initialiseur :

- lorsque seuls ``num_electrons`` et ``num_spatial_orbitals`` sont spécifiés, ces entiers indiquent respectivement le nombre d’électrons actifs et d’orbitales. L'espace actif sera puis être choisi autour du niveau de Fermi, ce qui donne un choix unique pour n'importe quelle paire de nombres. Néanmoins, les critères suivants doivent être remplis :

   * le nombre restant d'électrons inactifs doit être un nombre pair et positif;
   * le nombre d'orbitales actives ``num_spatial_orbitals`` ne doit pas dépasser le nombre total d'orbitales moins le nombre d'orbitales occupées par les électrons inactifs;

- lorsque ``num_electrons`` est un tuple, cela doit indiquer le nombre de spins alpha et bêta électrons, respectivement. Les mêmes exigences que celles énumérées précédemment doivent être remplies;
- enfin, il est possible de sélectionner un ensemble personnalisé d'orbitales actives via leurs indices en utilisant ``active_orbitals``. Ceci permet de sélectionner un espace actif qui n'est pas placé autour du niveau de Fermi comme décrit dans le premier cas ci-dessus. Lorsque vous utilisez cet argument de mot-clé, les critères suivants doivent être remplis *en plus* de ceux énumérés ci-dessus :

   * la longueur de `active_orbitals` doit être égale à ``num_spatial_orbitals``. Noter que qiskit-nature de  **déduit pas** le nombre d'orbitales actives à partir de cette liste d'indices !

   * lors de l'utilisation d'un tuple de listes pour indiquer les indices orbitaux de spin alpha et bêta séparément, les deux listes doivent remplir le critère précédent;

   * le plus grand indice orbital ne peut **pas** dépasser le ``num_spatial_orbitals`` disponible.

In [66]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

# We want to reduce it to an active space of 4 electrons in 4 orbitals (HOMO-1,HOMO, LUMO, LUMO+1)
as_transformer = ActiveSpaceTransformer(4, 4) # ActiveSpaceTransformer(num_electrons, num_spatial_orbitals)

H2O_as_problem = as_transformer.transform(H2O_problem)
print(H2O_as_problem.num_particles)
print(H2O_as_problem.num_spatial_orbitals)

(2, 2)
4


`qiskit_nature.second_q.transformers.ActiveSpaceTransformer` permet aussi de spécifier manuellement les indices des orbitales actives. Cela vous permet de sélectionner manuellement des _active space_ qui ne se trouvent pas en permanence autour du niveau de Fermi.

In [67]:
as_transformer = ActiveSpaceTransformer(2, 2, active_orbitals=[3, 6])

H2O_as_problem = as_transformer.transform(H2O_problem)
print(H2O_as_problem.num_particles)
print(H2O_as_problem.num_spatial_orbitals)

(1, 1)
2


### Energie fondamentale de la molecule 1,2-Thiazole atom

Comme vous le savez déjà, il est difficile d'obtenir le Hamiltonien qubit de cette molecule, au vue de sa taille. Nous allons donc utiliser `ActiveSpaceTransformer` pour réduire sa taille.

In [68]:
# Molecular structure
driver = PySCFDriver(
    atom="""C      1.1291      0.0795     -0.5259
  C      0.7115     -1.2207     -0.4748
  H      2.0789      0.4171     -0.9381
  H      1.2719     -2.0822     -0.8377
  S     -0.0500      1.1306      0.1514
  N     -1.1147     -0.1822      0.5074
  C     -0.5926     -1.3347      0.1299
  H     -1.1422     -2.2662      0.2851""", #C1,2-Thiazole atom
    basis='STO-3G'
)
# Electronic structure problem
problem = driver.run()

In [69]:
# Some properties
print(f"Molecule, basis: {driver.basis}, Hartree-Fock calculation")
print(f"Number of alpha electrons: {problem.num_alpha}")
print(f"Number of beta electrons: {problem.num_beta}")
print(f"Number of spin orbitals: {problem.num_spin_orbitals}")
print(f"Spin orbitals occupation: {problem.orbital_occupations}")
print(f"Spin orbitals energies: {problem.orbital_energies}")
print(f"Molecule reference (HF) total energy: {problem.reference_energy} Ha")
print(f"Molecule nuclear repulsion energy: {problem.nuclear_repulsion_energy} Ha")


Molecule, basis: STO-3G, Hartree-Fock calculation
Number of alpha electrons: 22
Number of beta electrons: 22
Number of spin orbitals: 64
Spin orbitals occupation: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Spin orbitals energies: [-90.84544727 -15.32141747 -11.06407434 -11.04110113 -11.02753474
  -8.73707496  -6.36878764  -6.36729122  -6.3646857   -1.19448958
  -0.98941296  -0.92735272  -0.72288664  -0.69941879  -0.63361482
  -0.51551986  -0.48793488  -0.48191303  -0.40782084  -0.34186567
  -0.30009975  -0.27940707   0.23917557   0.37822271   0.38660052
   0.50039675   0.61283911   0.69130018   0.76619362   0.824971
   0.90816777   1.04637883]
Molecule reference (HF) total energy: -560.8391624792079 Ha
Molecule nuclear repulsion energy: 203.5391249740159 Ha


In [70]:
# Fermi Level and properties
from pyscf.data import nist
au2ev = nist.HARTREE2EV

print(f"HOMO and LUMO index are {problem.num_alpha-1, problem.num_alpha}")
print(f"HOMO and LUMO energies in eV are\
{problem.orbital_energies[problem.num_alpha-1],problem.orbital_energies[problem.num_alpha] * au2ev}")
print(f"HOMO-LUMO gap in eV is \
{abs(problem.orbital_energies[problem.num_alpha-1]-problem.orbital_energies[problem.num_alpha]) * au2ev}")

HOMO and LUMO index are (21, 22)
HOMO and LUMO energies in eV are(-0.2794070652303571, 6.508298765683214)
HOMO-LUMO gap in eV is 14.111352274381781


In [71]:
problem.num_particles

(22, 22)

* Définition d'un espace actif autour du niveau de Fermi

Nous allons choisir un espace actif avec 4 orbitale spatiale, c'est-à-dire (HOMO-1,HOMO, LUMO, LUMO+1).

In [72]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
problem = ActiveSpaceTransformer(4, 4).transform(problem) # Utiliser (2,2) pour rendre le calcul moins couteux (+perte précision)

(2, 2)

On note que nous sommes partis de (22,22) à (2,2) particules alpha et beta!

In [73]:
hamiltonian = problem.hamiltonian # Hamiltonien en 1ere quantification
fermionic_op = hamiltonian.second_q_op() # Hamiltonien fermionique
if len(fermionic_op) <= 20:
    print(fermionic_op)
else: # print the first 20 terms of the fermionic Hamiltonian operator of the molecule
    print("\n".join(str(fermionic_op).splitlines()[:22] + ["..."]))

Fermionic Operator
number spin orbitals=8, number terms=1056
  0.18037101450036708 * ( +_0 +_0 -_0 -_0 )
+ -0.003998983948927016 * ( +_0 +_0 -_1 -_0 )
+ -0.003708063437679914 * ( +_0 +_0 -_2 -_0 )
+ -0.0033104464651577087 * ( +_0 +_0 -_3 -_0 )
+ -0.003998983948927016 * ( +_0 +_1 -_0 -_0 )
+ 0.15726000430579692 * ( +_0 +_1 -_1 -_0 )
+ 0.0018142693697536605 * ( +_0 +_1 -_2 -_0 )
+ 0.016408915168517547 * ( +_0 +_1 -_3 -_0 )
+ -0.003708063437679914 * ( +_0 +_2 -_0 -_0 )
+ 0.0018142693697536605 * ( +_0 +_2 -_1 -_0 )
+ 0.17784258943634249 * ( +_0 +_2 -_2 -_0 )
+ 0.01716541288730581 * ( +_0 +_2 -_3 -_0 )
+ -0.0033104464651577087 * ( +_0 +_3 -_0 -_0 )
+ 0.016408915168517547 * ( +_0 +_3 -_1 -_0 )
+ 0.01716541288730581 * ( +_0 +_3 -_2 -_0 )
+ 0.16914164309672147 * ( +_0 +_3 -_3 -_0 )
+ 0.18037101450036708 * ( +_0 +_4 -_4 -_0 )
+ -0.003998983948927016 * ( +_0 +_4 -_5 -_0 )
+ -0.003708063437679914 * ( +_0 +_4 -_6 -_0 )
+ -0.0033104464651577087 * ( +_0 +_4 -_7 -_0 )
...


* Hamiltonien qubit avec la réduction $\mathbb{Z}_2$

In [74]:
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper, BravyiKitaevMapper, TaperedQubitMapper

mapper = ParityMapper(num_particles=problem.num_particles)
mapper = problem.get_tapered_mapper(mapper)
Hamil_z2qubit = mapper.map(fermionic_op)

print(f"Number of items in the PM Z2 Pauli list:", len(Hamil_z2qubit))
if len(Hamil_z2qubit) <= 10:
    print(Hamil_z2qubit)
else:
    print(Hamil_z2qubit[0:10])

Number of items in the PM Z2 Pauli list: 325
SparsePauliOp(['IIIIII', 'IIIIIZ', 'IIIIZZ', 'IIIIZI', 'IIIZXZ', 'IIIZXI', 'IIIIXI', 'IIIIXZ', 'IIIXXZ', 'IIIXXI'],
              coeffs=[-2.07212264e+00+0.j,  7.84078941e-02+0.j,  8.19933172e-02+0.j,
  1.26938405e-01+0.j, -1.51674797e-02+0.j,  1.99548286e-03+0.j,
  1.51674797e-02+0.j, -1.99548286e-03+0.j,  1.24056296e-02+0.j,
  8.26189392e-03+0.j])


* Circuit de l'état initial

In [75]:
from qiskit_nature.second_q.circuit.library import HartreeFock

HF_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
HF_state.draw('text',initial_state=True)

┌───┐
q_0: |0>┤ X ├
        └───┘
q_1: |0>─────
             
q_2: |0>─────
        ┌───┐
q_3: |0>┤ X ├
        └───┘
q_4: |0>─────
             
q_5: |0>─────

Ce qui précède est une solution au **Devoir sur le Hamiltonien Qubit**.

* Définisons le solveur VQE.

In [76]:
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import SLSQP
from qiskit_algorithms import VQE
import numpy as np

ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state = HF_state)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = np.zeros(ansatz.num_parameters) # Initialisation with zero-qubit

* Calcul et résultats

In [77]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# Résultats VQE
GS_VQE = GroundStateEigensolver(mapper, vqe_solver)
GS_VQE_res = GS_VQE.solve(problem)

In [78]:
# Resultats de la diagonalisation exacte
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
NP_calc = GroundStateEigensolver(mapper, numpy_solver)
GS_NP_res = NP_calc.solve(problem)

### Class `qiskit_nature_pyscf.PySCFGroundStateSolver`

Nous allons maintenant utiliser le plugin `qiskit_nature_pyscf` qui couple PySCF et Qiskit Nature.  C'est un solveur [FCI](https://en.wikipedia.org/wiki/Full_configuration_interaction) (Full Configuration Interaction) basé sur Qiskit-Nature qui permet à un utilisateur de PySCF (Python-based Simulations of Chemistry Framework) de tirer parti des algorithmes quantique implémentés dans Qiskit-Nature pour être utilisés à la place de leurs homologues classiques (dans un esprit similaire à l'intégration NWChemEx).

La classe `qiskit_nature_pyscf.PySCFGroundStateSolver` s'appuie sur le module ``fci`` de PySCF. Il n'utilise aucun algorithmes quantiques (puisqu'il les remplace dans le workflow de Qiskit-Naure) mais fournit à la place un utilitaire pour déboguer les workflows de calcul classique basés sur Qiskit-Nature.

Plus important encore, il fournit une implémentation plus efficace de ce que Qiskit-Nature réalise en utilisant la classe `qiskit_algorithms.NumPyMinimumEigensolver` en combinaison avec un ``filter_criterion``. Pour les états fondamentaux de spin autres que le singlet, l'utilisation des composants Qiskit-Nature est beaucoup plus complexe, alors que cette classe fournit une alternative facile à utiliser.

In [79]:
from pyscf import fci
try:
    from qiskit_nature_pyscf import PySCFGroundStateSolver
except:
    %pip install qiskit-nature-pyscf
    from qiskit_nature_pyscf import PySCFGroundStateSolver

au2kcalc = 627.509474

In [80]:
fci_solver = fci.direct_uhf.FCI()
solver_pyscf = PySCFGroundStateSolver(fci_solver)

GS_FCI_res = solver_pyscf.solve(problem)
print(GS_FCI_res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -764.419565104202
  - computed part:      -3.079445017779
  - ActiveSpaceTransformer extracted energy part: -761.340120086423
~ Nuclear repulsion energy (Hartree): 203.539124974016
> Total ground state energy (Hartree): -560.880440130186
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000


In [81]:
print(f'error between the two exact calculations, {abs(GS_FCI_res.total_energies[0] - GS_NP_res.total_energies[0]) * au2kcalc} kcal/mol')

error between the two exact calculations, 0.0 kcal/mol


### Visualisation des résultats

In [82]:
error_VQE_NP = abs(GS_VQE_res.total_energies[0] - GS_NP_res.total_energies[0]) * au2kcalc
error_HF_NP = abs(GS_VQE_res.hartree_fock_energy - GS_NP_res.total_energies[0]) * au2kcalc

import pandas as pd

# Create the results dataframe
dict_res = {'FCI': [GS_FCI_res.total_energies[0], ''],
            'Numpy': [GS_NP_res.total_energies[0], ''],
            'VQE': [GS_VQE_res.total_energies[0], error_VQE_NP],
            'HF': [GS_VQE_res.hartree_fock_energy, error_HF_NP]}
df_GS = pd.DataFrame(dict_res, index = ['E_tot (a.u.)', 'Error (kcal/mol)'])

df_GS


,FCI,Numpy,VQE,HF
E_tot (a.u.),-560.88044,-560.88044,-560.880213,-560.839162
Error (kcal/mol),,,0.142459,25.902117


## Etats excités

Calculons maintenant les énergies des états excités de notre Hamiltonien moléculaire et en déduisons la
* la bande interdite ou l'écart entre le niveau fondamental $S_0$ et le premier niveau excité (le niveau triplet $T_1$),
* et et de l'énergie de fluorescence ou l'écart entre le niveau fondamental $S_0$ et le premier niveau excité singulet $S_1$.

![Molecule_HOMO-LUMO_diagram](Graphics/Molecule_HOMO-LUMO_diagram.png)


### Calcul avec `ExcitedStatesEigensolver`

Effectuons les calculs avec `NumPyEigensolver` avec le critère de filtre par défaut activé.

In [83]:
from qiskit_nature.second_q.algorithms import ExcitedStatesEigensolver
from qiskit_algorithms import NumPyEigensolver

numpy_solver = NumPyEigensolver(k=4, filter_criterion=problem.get_default_filter_criterion())

numpy_ES_solver = ExcitedStatesEigensolver(mapper, numpy_solver)
NP_ES = numpy_ES_solver.solve(problem)

print(NP_ES)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -764.419565104202
  - computed part:      -3.079445017779
  - ActiveSpaceTransformer extracted energy part: -761.340120086423
~ Nuclear repulsion energy (Hartree): 203.539124974016
> Total ground state energy (Hartree): -560.880440130186
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -764.122470836677
> Total excited state energy (Hartree): -560.583345862661
  2: 
* Electronic excited state energy (Hartree): -764.043946955492
> Total excited state energy (Hartree): -560.504821981476
  3: 
* Electronic excited state energy (Hartree): -764.009962482753
> Total excited state energy (Hartree): -560.470837508737
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
  2:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
  3:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ==

In [84]:
ET1_NP = NP_ES.total_energies[1]
ES1_NP = NP_ES.total_energies[2]
NP_gap = NP_ES.computed_energies[1] * au2ev
NP_f_energy = NP_ES.computed_energies[2] * au2ev

print(f'Total Numpy ES energy T1 = {ET1_NP} a.u.')
print(f'Total Numpy ES energy S1 = {ES1_NP} a.u.')
print(f'The bandgap obtained form Numpy ES calculations is : {NP_gap} eV')
print(f'The fluorescence energy obtained form Numpy ES calculations is : {NP_f_energy} eV')


Total Numpy ES energy T1 = -560.5833458626611 a.u.
Total Numpy ES energy S1 = -560.504821981476 a.u.
The bandgap obtained form Numpy ES calculations is : -75.71162030819806 eV
The fluorescence energy obtained form Numpy ES calculations is : -73.574876665481 eV


In [85]:
# Create the results dataframe
list_results_NP = [ET1_NP, ES1_NP, NP_f_energy, NP_gap]
dict_results_NP = {'Numpy ES': list_results_NP}
df_NP = pd.DataFrame(dict_results_NP,
                    index = ['ES energy T1 (a.u.)', 'ES energy S1 (a.u.)',
                            'f_energy ES1-ES0 (eV)', 'Gap ES1 - ET1 (eV)'])

df_NP

,Numpy ES
ES energy T1 (a.u.),-560.583346
ES energy S1 (a.u.),-560.504822
f_energy ES1-ES0 (eV),-73.574877
Gap ES1 - ET1 (eV),-75.711620


### Calculs avec `PySCFGroundStateSolver`

In [86]:
fci_solver.nroots = 4
solver_fci = PySCFGroundStateSolver(fci_solver)
FCI_ES = solver_fci.solve(problem)
print(FCI_ES)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -764.419565104202
  - computed part:      -3.079445017779
  - ActiveSpaceTransformer extracted energy part: -761.340120086423
~ Nuclear repulsion energy (Hartree): 203.539124974016
> Total ground state energy (Hartree): -560.880440130186
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -764.251287516913
> Total excited state energy (Hartree): -560.712162542897
  2: 
* Electronic excited state energy (Hartree): -764.201014546775
> Total excited state energy (Hartree): -560.661889572759
  3: 
* Electronic excited state energy (Hartree): -764.122470836677
> Total excited state energy (Hartree): -560.583345862661
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
  1:  # Particles: 4.000 S: 1.000 S^2: 2.000 M: 0.000
  2:  # Particles: 4.000 S: 1.000 S^2: 2.000 M: 0.000
  3:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000


In [87]:
ET1_FCI = FCI_ES.total_energies[1]
ES1_FCI = FCI_ES.total_energies[2]
FCI_gap = FCI_ES.computed_energies[1] * au2ev
FCI_f_energy = FCI_ES.computed_energies[2] * au2ev

print(f'Total FCI ES energy T1 = {ET1_FCI} a.u.')
print(f'Total FCI ES energy S1 = {ES1_FCI} a.u.')
print(f'The bandgap obtained form FCI ES calculations is : {FCI_gap} eV')
print(f'The fluorescence energy obtained form FCI ES calculations is : {FCI_f_energy} eV')

Total FCI ES energy T1 = -560.7121625428969 a.u.
Total FCI ES energy S1 = -560.6618895727594 a.u.
The bandgap obtained form FCI ES calculations is : -79.21690071990727 eV
The fluorescence energy obtained form FCI ES calculations is : -77.84890352312551 eV


In [88]:
# Create the results dataframe
list_results_FCI = [ET1_FCI, ES1_FCI, FCI_f_energy, FCI_gap]
dict_results_FCI = {'FCI ES': list_results_FCI}
df_FCI = pd.DataFrame(dict_results_FCI,
                    index = ['ES energy T1 (a.u.)', 'ES energy S1 (a.u.)',
                            'f_energy ES1-ES0 (eV)', 'Gap ES1 - ET1 (eV)'])

df_FCI

,FCI ES
ES energy T1 (a.u.),-560.712163
ES energy S1 (a.u.),-560.661890
f_energy ES1-ES0 (eV),-77.848904
Gap ES1 - ET1 (eV),-79.216901


### Calculs avec `QEOM`

 Puisque nous avons déjà défini le système, nous avons besoin d'accéder à l'énergie d'excitation en utilisant l'[algorithme quantique d'équation du mouvement (qEOM)](https://arxiv.org/abs/1910.12890).

La classe `qiskit_nature.second_q.algorithms.QEOM` implémente cet algorithme qui approxime les propriétés de l'état excité d'un problème en utilisant des mesures supplémentaires sur l'état fondamental fournies par un objet `GroundStateSolver`. La précision de la méthode `GroundStateSolver.solve` pour l'approximation de l'état fondamental affecte directement la précision de l'algorithme qEOM pour le même problème. Les excitations sont utilisées pour construire un sous-espace linéaire dans lequel un problème de valeurs propres pour l'Hamiltonien projeté sera résolu. Cette méthode fonctionne généralement bien pour calculer les états excités les plus bas d'un problème. Les énergies des états excités sont calculées par défaut dans cet algorithme pour tous les états excités.

In [89]:
from qiskit_nature.second_q.algorithms import QEOM
from qiskit_nature.second_q.algorithms.excited_states_solvers.qeom import EvaluationRule

qeom_ES_solver = QEOM(GS_VQE, Estimator(), "sd", EvaluationRule.ALL)
qeom_ES = qeom_ES_solver.solve(problem)
print(qeom_ES)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -764.419338081372
  - computed part:      -3.079217994949
  - ActiveSpaceTransformer extracted energy part: -761.340120086423
~ Nuclear repulsion energy (Hartree): 203.539124974016
> Total ground state energy (Hartree): -560.880213107357
 
=== EXCITED STATE ENERGIES ===
 
  1: 
* Electronic excited state energy (Hartree): -764.25038974753
> Total excited state energy (Hartree): -560.711264773515
  2: 
* Electronic excited state energy (Hartree): -764.199095721532
> Total excited state energy (Hartree): -560.659970747516
  3: 
* Electronic excited state energy (Hartree): -764.115515889976
> Total excited state energy (Hartree): -560.57639091596
  4: 
* Electronic excited state energy (Hartree): -764.089515004816
> Total excited state energy (Hartree): -560.5503900308
  5: 
* Electronic excited state energy (Hartree): -764.065618163307
> Total excited state energy (Hartree): -560.526493189291
  6: 
* Electronic exc

In [90]:
ET1 = qeom_ES.total_energies[1]
ES1 = qeom_ES.total_energies[2]
qeom_gap = qeom_ES.computed_energies[1] * au2ev
qeom_f_energy = qeom_ES.computed_energies[2] * au2ev

print(f'Total QEOM ES energy T1 = {ET1} a.u.')
print(f'Total QEOM ES energy S1 = {ES1} a.u.')
print(f'The bandgap obtained form Py ES calculations is : {qeom_gap} eV')
print(f'The fluorescence energy obtained form Py ES calculations is : {qeom_f_energy} eV')

Total QEOM ES energy T1 = -560.7112647735146 a.u.
Total QEOM ES energy S1 = -560.6599707475164 a.u.
The bandgap obtained form Py ES calculations is : -79.19247117068878 eV
The fluorescence energy obtained form Py ES calculations is : -77.79668962873295 eV


In [91]:
# Create the results dataframe
list_results_QEOM = [ET1, ES1, qeom_f_energy, qeom_gap]
dict_results_QEOM = {'QEOM': list_results_QEOM}
df_QEOM = pd.DataFrame(dict_results_QEOM,
                    index = ['ES energy T1 (a.u.)', 'ES energy S1 (a.u.)',
                            'f_energy ES1-ES0 (eV)', 'Gap ES1 - ET1 (eV)'])

df_QEOM

,QEOM
ES energy T1 (a.u.),-560.711265
ES energy S1 (a.u.),-560.659971
f_energy ES1-ES0 (eV),-77.796690
Gap ES1 - ET1 (eV),-79.192471


### Synthèse des résultats sur les états excités

In [92]:
# Visualisation of results

# Create the results dataframe
dict_res = {'FCI ES': list_results_FCI, 'Numpy ES': list_results_NP,
            'QEOM': list_results_QEOM
            }
df_ES = pd.DataFrame(dict_res,
                     index = ['ES energy T1 (a.u.)', 'ES energy S1 (a.u.)',
                            'f_energy ES1-ES0 (eV)', 'Gap ES1 - ET1 (eV)'])

df_ES


,FCI ES,Numpy ES,QEOM
ES energy T1 (a.u.),-560.712163,-560.583346,-560.711265
ES energy S1 (a.u.),-560.661890,-560.504822,-560.659971
f_energy ES1-ES0 (eV),-77.848904,-73.574877,-77.796690
Gap ES1 - ET1 (eV),-79.216901,-75.711620,-79.192471
